In [43]:
!pip install facenet_pytorch

In [44]:
!pip install matplotlib

In [45]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn

import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob
import os
# from tensorflow.keras.models import load_model

# from tensorflow.keras.models import Model
import os
import warnings


from torch.utils.data import random_split
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim



In [46]:
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=FutureWarning) 
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:1


In [47]:
# data = pd.read_csv('/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Data/IJB_C/labels.csv')
# id_mapping = {}
# def extractId(row):
#     id = row['SUBJECT_ID']

#     if id not in id_mapping:
#         id_mapping[id] = len(id_mapping)
    
#     return id_mapping[id]

In [48]:
# data["NAME"] = data.apply(extractId, axis=1)
# print(len(id_mapping))

In [49]:
# data.head()

In [50]:
# data.to_csv("/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Data/IJB_C/labels_id.csv", index=False)

In [51]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.gender_mapping = {'males': 0, 'females': 1}
        self.ethnicity_mapping = {'indian': 0, 'white': 1, 'asian': 2, 'black': 3}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = "/home/csgrad/byalavar/FHE/HEAAN/FG_Round_2/IJB_C/data/" + str(self.data.iloc[idx]['TEMPLATE_ID']) + ".jpg"

        image = Image.open(img_path).convert('RGB')

        #print(img_path)
        gender = self.data.iloc[idx]['GENDER']
        ethnicity = self.data.iloc[idx]['SKIN_COLOUR']
        age = self.data.iloc[idx]['AGE']
        id = self.data.iloc[idx]['NAME']
        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        label = {
            'Name': id,
            'age': age,
            'gender': gender,
            'ethnicity': ethnicity
        
        }
        return image,label

# Example usage:
csv_file_path = '/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Data/IJB_C/labels_id.csv'  # Replace with the actual path to your CSV file
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the image to the desired size
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


dataset = CustomDataset(csv_file_path, transform=transform)

In [52]:
temp = pd.read_csv(csv_file_path)
print("Identity Classes : ", temp['NAME'].nunique())
print("Age Classes : ", temp['AGE'].nunique())
print("Gender Classes : ", temp['GENDER'].nunique())
print("Ethnicity Classes : ", temp['SKIN_COLOUR'].nunique())

Identity Classes :  4297
Age Classes :  3
Gender Classes :  2
Ethnicity Classes :  2


In [53]:
temp['AGE'].value_counts()

AGE
1    64152
0    24155
2    11879
Name: count, dtype: int64

In [54]:
temp['GENDER'].value_counts()

GENDER
0    63735
1    36451
Name: count, dtype: int64

In [55]:
temp['SKIN_COLOUR'].value_counts()

SKIN_COLOUR
0    80645
1    19541
Name: count, dtype: int64

In [56]:
temp = pd.read_csv(csv_file_path)
#plt.hist(temp['Ag'], bins=2, color='blue', edgecolor='black')
label_counts = torch.bincount(torch.tensor(temp['AGE']))
print(label_counts)
total_samples = label_counts.sum().float()
class_weights = total_samples / (label_counts + 1e-7)
ageWeights =class_weights/ class_weights.sum()


tensor([24155, 64152, 11879])


In [57]:
ageWeights

tensor([0.2933, 0.1104, 0.5963])

In [58]:
temp = pd.read_csv(csv_file_path)
#plt.hist(temp['Ag'], bins=2, color='blue', edgecolor='black')
label_counts = torch.bincount(torch.tensor(temp['GENDER']))
total_samples = label_counts.sum().float()
class_weights = total_samples / (label_counts + 1e-7)
genderWeights =class_weights/ class_weights.sum()

In [59]:
temp = pd.read_csv(csv_file_path)
#plt.hist(temp['Ag'], bins=2, color='blue', edgecolor='black')
label_counts = torch.bincount(torch.tensor(temp['SKIN_COLOUR']))
total_samples = label_counts.sum().float()
class_weights = total_samples / (label_counts + 1e-7)
ethnWeights =class_weights/ class_weights.sum()

In [60]:
ageWeights

tensor([0.2933, 0.1104, 0.5963])

In [61]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create dataloaders for training and testing sets
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [62]:
tempIter = iter(trainloader)
images,labels = next(tempIter)
print(labels)

{'Name': tensor([2678,  254, 3856,  574,  681, 1195,  412, 1650, 1301, 2077, 2013,   45,
        3100,  252, 1002, 1446,  691,  847, 1466,  911,   43,   43,  341,  726,
        1493,   43, 2264, 1222, 3668,   29, 1611, 1274]), 'age': tensor([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0]), 'gender': tensor([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 1, 0]), 'ethnicity': tensor([0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 0])}


In [63]:
def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

In [64]:
from torch.nn.modules import MSELoss
from torch.nn.modules.loss import CrossEntropyLoss
from torch.nn.parallel import DataParallel

class GenderDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )
        
    def forward(self, x):
        return self.layer(x)

class AgeDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size, 32),
            nn.ReLU(),
            nn.Linear(32, 3)
        )
        
    def forward(self, x):
        return self.layer(x)
    
class EthnicityDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
        
    def forward(self, x):
        return self.layer(x)
    
class IdentityDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size, 128), 
            nn.ReLU(), 
            nn.Linear(128, 512),
            nn.ReLU(), 
            nn.Linear(512, 1024),
            nn.ReLU(), 
            nn.Linear(1024, 4297))
        
    def forward(self, x):
        return self.layer(x)
    

class Matryoshka_CE_Loss(nn.Module):
    def __init__(self, relative_importance = 1.0):
        super(Matryoshka_CE_Loss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()
        self.relative_importance = relative_importance # usually set to all ones
    
    def forward(self, output, target):
        loss = 0
        for i in range(len(output)):
            # loss += self.relative_importance[i] * self.criterion(output[i], target)
            loss += self.relative_importance * self.criterion(output[i], target)
        return loss

class MRL_Linear_Layer(nn.Module):
    def __init__(self, nesting_list, num_classes=1000, efficient=False, **kwargs):
        super(MRL_Linear_Layer, self).__init__()
        self.nesting_list=nesting_list
        self.num_classes=num_classes # Number of classes for classification
        self.efficient = efficient
        if self.efficient:
            setattr(self, f"nesting_classifier_{0}", nn.Linear(nesting_list[-1], self.num_classes, **kwargs))		
        else:	
            for i, num_feat in enumerate(self.nesting_list):
                setattr(self, f"nesting_classifier_{i}", nn.Linear(num_feat, self.num_classes, **kwargs))	


    def forward(self, x):
        nesting_logits = ()
        for i, num_feat in enumerate(self.nesting_list):
            if self.efficient:
                if self.nesting_classifier_0.bias is None:
                    nesting_logits+= (torch.matmul(x[:, :num_feat], (self.nesting_classifier_0.weight[:, :num_feat]).t()), )
                else:
                    nesting_logits+= (torch.matmul(x[:, :num_feat], (self.nesting_classifier_0.weight[:, :num_feat]).t()) + self.nesting_classifier_0.bias, )
            else:
                nesting_logits +=  (getattr(self, f"nesting_classifier_{i}")(x[:, :num_feat]),)

        return nesting_logits

class MRL_Model(nn.Module):
    def __init__(self, compression_size, num_classes, efficient=False):
        super(MRL_Model, self).__init__()
        self.ll1 = MRL_Linear_Layer([256], num_classes = compression_size, efficient = efficient)
        self.ll2 = MRL_Linear_Layer([compression_size], num_classes = num_classes, efficient = efficient)
        # self.flatten = nn.Flatten()
        
    def forward(self, x):
        # x = self.flatten(x)
        x = self.ll1(x)
        return self.ll2(x[0]), x
    

class CompressionLoss(nn.Module):
    def __init__(self, batch_size):
        super(CompressionLoss, self).__init__()
        self.batch_size = batch_size

    def forward(self, img, genuine_imgs, imposter_imgs, genuine_sim, imposter_sim):
        # img -> base image embedding
        # genuine_imgs, imposter_imgs -> Compressed embeddings of dimension 128
        # genuine_sim, imposter_sim -> cosine smilairty scores of uncompressed embeddings of dimension 512


        # Compute cosine similarity loss
        genuine_loss = 0
        for i in range(len(genuine_imgs)):
            sim_score = F.cosine_similarity(img, genuine_imgs[i])
            genuine_loss += torch.mean(torch.abs(sim_score - genuine_sim[i].view(-1)))

        imp_loss = 0
        for i in range(len(imposter_imgs)):
            imp_sim_score = F.cosine_similarity(img, imposter_imgs[i])
            imp_loss += torch.mean(torch.abs(imp_sim_score - imposter_sim[i].view(-1)))

        
        # Compute Covariance Loss
        # matrix -> Concatenates compressed genuine and imposter embeddings
        
        matrix = torch.cat([genuine_imgs[0], imposter_imgs[0]], dim=0)
        for i in range(1, len(genuine_imgs)):
            matrix = torch.cat([matrix, genuine_imgs[i]], dim = 0)
            matrix = torch.cat([matrix, imposter_imgs[i]], dim = 0)


        mean_matrix = torch.mean(matrix, dim = 0)
        mx = torch.matmul(mean_matrix.t(), mean_matrix)
        vx = torch.matmul(matrix.t(), matrix) / self.batch_size # Dividing by batch size as done here https://github.com/human-analysis/hers-encrypted-image-search/blob/master/deep_mds%2B%2B/nntools/tensorflow/networks/deepmds.py#L94
        cov_matrix = mx - vx
        diag = torch.diag(cov_matrix.diag())
        cov_loss = torch.mean(torch.abs(cov_matrix - diag))
        
        # Compute Supervise Loss - Not used for ArcFace
        # # g_dist -> Concatenates compressed genuine embeddings
        # g_dist = torch.cat([genuine_imgs[0], genuine_imgs[1]], dim = 0)
        # for i in range(2, len(genuine_imgs)):
        #     g_dist = torch.cat([g_dist, genuine_imgs[i]], dim = 0)

        # # g_dist -> Concatenates compressed imposter embeddings
        # imp_dist = torch.cat([imposter_imgs[0], imposter_imgs[1]], dim = 0)
        # for i in range(2, len(imposter_imgs)):
        #     imp_dist = torch.cat([imp_dist, imposter_imgs[i]], dim = 0)

        # gloss = (torch.mean(g_dist) + 1.0) * 0.5
        # iloss = (torch.mean(imp_dist) + 1.0) * 0.5

        # lda_term = iloss / gloss
        # return genuine_loss + imp_loss + 10 * cov_loss 
        return 10*genuine_loss + imp_loss + cov_loss


In [65]:

class faceAnalytics(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.layer1=nn.Linear(128,64)
        self.dropout1=nn.Dropout(0.2)
        self.layer2=nn.Linear(64,32)
        #self.layer3=nn.Linear(1024,512)
        # self.layer4=nn.Linear(128,64)
        self.dropout2=nn.Dropout(0.2)
        self.genderOut = nn.Sequential(
            nn.Linear(128,64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )
        self.ageOut = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,4)
        )
        self.ethnicityOut = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )

        self.identity = nn.Sequential(
            nn.Linear(128,64), 
            nn.ReLU(), 
            nn.Linear(64,80))
        # self.maxVal = 0
        # self.min=0
        
    
    def writeResult(self,result):
       output_directory=""
       file_name = "resultAge.txt"

       with open(os.path.join(output_directory, file_name), "w") as file:
        for value in result:
            file.write(f"{value}\n")
    
    def forward(self,x):

        #print("Input",x[0])
        # x=self.layer1(x)
        # x = nn.ReLU()(x)
        # x=self.layer2(x)
        # x = nn.ReLU()(x)

        gender = self.genderOut(x)
        age = self.ageOut(x)
        ethn = self.ethnicityOut(x)
        id = self.identity(x)
        # identity = self.identity(x)
        return gender, age, ethn, id
    
    
    def are_values_in_same_class(self,value1, value2):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]
    
    # Check if both values fall into the same class range
        for class_range in class_ranges:
            if class_range[0] <= value1 < class_range[1] and class_range[0] <= value2 < class_range[1]:
                return True
    
        return False    
    
    def trainMRL(self, compressed_dim, trainloader, resnet, MRL_Model, MRL_Loss, device, episodes):
        # input_dim = 512
        # compressed_dim = 128
        model = MRL_Model
        # Define optimizer and loss function
        
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        MRLoss = MRL_Loss.to(device)
        model.train()
        for ep in range(episodes):
            total_loss = 0
            print("Starting " + str(ep))

            for img, data in trainloader:
                inputs = img.to(device = device)
                label = data['Name'].to(device=device)    
                # inputs = inputs.detach().numpy()
                # print(inputs.shape)
                # inputs = np.transpose(inputs, (0, 2, 3, 1))
                # inputs = torch.from_numpy(inputs).to(device)
                # print(inputs.shape)
                embeddings = resnet(inputs)
                output, _ = model(embeddings)
                # print()
                loss = MRLoss(output, label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"Epoch [{ep + 1}/{episodes}], Loss: {total_loss}")
        torch.save(model, f'MRL_IJB_C_512x{compressed_dim}_FaceNet_{episodes}_2.pt')
        print("Training complete!")

    def testMRL(self, trainloader, testloader, resnet, MRL_Model, idModel, genderModel, ageModel, ethModel, device, episodes):
        # self.train()
        learningRate = 0.001
        gender_loss = nn.CrossEntropyLoss().to(device) 
        age_loss = nn.CrossEntropyLoss().to(device) 
        ethn_loss = nn.CrossEntropyLoss().to(device)
        identityLoss = nn.CrossEntropyLoss().to(device)
        IdOptimizer = torch.optim.Adam(idModel.parameters(), lr=learningRate)
        AgeOptimizer = torch.optim.Adam(ageModel.parameters(), lr=learningRate)
        GenderOptimizer = torch.optim.Adam(genderModel.parameters(), lr=learningRate)
        EthOptimizer = torch.optim.Adam(ethModel.parameters(), lr=learningRate)
        model = MRL_Model
        model.eval()
        idModel.train()
        genderModel.train()
        ageModel.train()
        ethModel.train()
        for ep in range(0, episodes):
            total_gender_loss = 0
            total_age_loss = 0
            total_ethn_loss = 0
            total_id_loss = 0
            for i, data in enumerate(trainloader):

                inputs = data[0].to(device = device)
                age_label = data[1]['age'].to(device=device)
                gender_label = data[1]['gender'].to(device=device)
                ethn_label = data[1]['ethnicity'].to(device=device)
                id_label = data[1]['Name'].to(device=device)
                    
                # inputs = inputs.detach().numpy()
                # inputs = np.transpose(inputs, (0, 2, 3, 1))
                # inputs = torch.from_numpy(inputs).to(device)
                #print(inputs.shape)
                embeddings = resnet(inputs)
                output, compressedOutput = model(embeddings)
                # print(output[0].shape)
                # print(compressedOutput[0].shape)
                # return
                output1 = torch.clone(compressedOutput[0])
                id = idModel(output1)
                output2 = torch.clone(compressedOutput[0])
                gender = genderModel(output2)
                output3 = torch.clone(compressedOutput[0])
                age = ageModel(output3)
                output4 = torch.clone(compressedOutput[0])
                eth = ethModel(output4)
                # gender, age, ethn, id = self(new_output)

                idLoss = identityLoss(id, id_label) 
                ageLoss = age_loss(age, age_label) 
                genLoss =  gender_loss(gender, gender_label)
                ethLoss = ethn_loss(eth, ethn_label) 
                
                IdOptimizer.zero_grad() 
                AgeOptimizer.zero_grad() 
                GenderOptimizer.zero_grad() 
                EthOptimizer.zero_grad()

                idLoss.backward(retain_graph = True)
                ageLoss.backward(retain_graph = True)
                genLoss.backward(retain_graph = True)
                ethLoss.backward(retain_graph = True) 
                
                IdOptimizer.step() 
                AgeOptimizer.step() 
                GenderOptimizer.step() 
                EthOptimizer.step()
                
                total_gender_loss += genLoss.item()
                total_age_loss += ageLoss.item()
                total_ethn_loss += ethLoss.item()
                total_id_loss += idLoss.item()
            print(f"Epoch [{ep + 1}/{episodes}], Id Loss: {total_id_loss}, Gender Loss: {total_gender_loss}, Age Loss: {total_age_loss}, Eth Loss: {total_ethn_loss} ")
        print("Training done!")

        # self.eval()
        count = 0
        genderAcc = 0
        ageAcc = 0
        ethnAcc = 0
        idAcc = 0
        for i, data in enumerate(testloader):

            inputs = data[0].to(device = device)
            age_label = data[1]['age'].to(device=device)
            gender_label = data[1]['gender'].to(device=device)
            ethn_label = data[1]['ethnicity'].to(device=device)
            id_label = data[1]['Name'].to(device=device)
            # inputs = inputs[:, [2, 1, 0], :, :]
            #print(inputs.shape)
            # inputs = inputs.detach().numpy()
            # inputs = np.transpose(inputs, (0, 2, 3, 1))
            # inputs = torch.from_numpy(inputs).to(device)
            embeddings = resnet(inputs)
            output, compressedOutput = model(embeddings)
            # gender, age, ethn, id = self(compressedOutput[0])

            output1 = torch.clone(compressedOutput[0])
            id = idModel(output1)
            output2 = torch.clone(compressedOutput[0])
            gender = genderModel(output2)
            output3 = torch.clone(compressedOutput[0])
            age = ageModel(output3)
            output4 = torch.clone(compressedOutput[0])
            eth = ethModel(output4)

            predictedGender = torch.argmax(gender, dim = 1)
            predictedEthn = torch.argmax(eth, dim = 1)
            predictedAge = torch.argmax(age, dim = 1)
            predictedId = torch.argmax(id, dim = 1)
            for j in range(0, predictedGender.shape[0]):
                count = count + 1
                if(predictedGender[j].item()==gender_label[j].item()):
                    genderAcc = genderAcc + 1
            
                if(predictedEthn[j].item()==ethn_label[j].item()):
                    ethnAcc = ethnAcc + 1

                if(predictedAge[j].item() == age_label[j].item()):
                    ageAcc = ageAcc + 1
                
                if(predictedId[j].item() == id_label[j].item()):
                    idAcc = idAcc + 1

        print("Id Accuracy : ", idAcc / count, "Gender Accuracy : ", genderAcc / count, " Age Acc : ", ageAcc / count, " ethnAcc : ", ethnAcc/count)
    
    
   

In [66]:
torch.cuda.device_count()
# torch.cuda.empty_cache()

4

In [67]:
batch_size = 32
FAmodel=faceAnalytics()
# nesting_list = [64, 128, 256]
# num_classes = 80
compression_size = 64
# MRL_model = MRL_Model(compression_size, num_classes=4297)
MRL_model = torch.load('/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Code/MRL/MRL_IJB_C_512x64_FaceNet_10.pt')
MRL_Loss = Matryoshka_CE_Loss()
FAmodel.to(device)
MRL_model.to(device)
resnet = InceptionResnetV1(pretrained='vggface2')
resnet.to(device)
resnet.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [68]:
FAmodel.trainMRL(compression_size, trainloader, resnet, MRL_model, MRL_Loss, device, 10)

Starting 0
Epoch [1/10], Loss: 15696.59893321991
Starting 1
Epoch [2/10], Loss: 15310.338696479797
Starting 2
Epoch [3/10], Loss: 14988.776918411255
Starting 3
Epoch [4/10], Loss: 14692.313131332397
Starting 4
Epoch [5/10], Loss: 14417.401333808899
Starting 5
Epoch [6/10], Loss: 14161.169349193573
Starting 6
Epoch [7/10], Loss: 13921.272170305252
Starting 7
Epoch [8/10], Loss: 13695.843700647354
Starting 8
Epoch [9/10], Loss: 13483.43609571457
Starting 9
Epoch [10/10], Loss: 13282.909888744354
Training complete!


In [69]:
idModel = IdentityDet(compression_size)
idModel.to(device)
ageModel = AgeDet(compression_size)
ageModel.to(device)
genderModel = GenderDet(compression_size)
genderModel.to(device)
ethModel = EthnicityDet(compression_size)
ethModel.to(device)
FAmodel.testMRL(trainloader, testloader, resnet, MRL_model, idModel, genderModel, ageModel, ethModel, device, 5)

Epoch [1/5], Id Loss: 15999.996017456055, Gender Loss: 1340.1081981658936, Age Loss: 1950.9275899231434, Eth Loss: 1146.7712873071432 
Epoch [2/5], Id Loss: 11291.283193826675, Gender Loss: 1274.4228852391243, Age Loss: 1893.979157358408, Eth Loss: 1114.0708124488592 
Epoch [3/5], Id Loss: 8789.205013751984, Gender Loss: 1253.528803884983, Age Loss: 1871.586389631033, Eth Loss: 1101.1664227098227 
Epoch [4/5], Id Loss: 6813.070330858231, Gender Loss: 1240.3868164420128, Age Loss: 1857.711944848299, Eth Loss: 1093.6773534715176 
Epoch [5/5], Id Loss: 5294.800201356411, Gender Loss: 1230.1653364151716, Age Loss: 1848.6985784471035, Eth Loss: 1087.7357434034348 
Training done!
Id Accuracy :  0.2847090527996806 Gender Accuracy :  0.7484279868250324  Age Acc :  0.6755165186146322  ethnAcc :  0.816947799181555


In [ ]:
compressed_outputs_list = []
for i, data in enumerate(trainloader):
    inputs = data[0].to(device=device)
    embeddings = resnet(inputs)
    output, compressedOutput = MRL_model(embeddings)
    compressed_output_np = compressedOutput[0].detach().cpu().numpy()
    compressed_outputs_list.append(compressed_output_np)

for i, data in enumerate(testloader):
    inputs = data[0].to(device=device)
    embeddings = resnet(inputs)
    output, compressedOutput = MRL_model(embeddings)
    compressed_output_np = compressedOutput[0].detach().cpu().numpy()
    compressed_outputs_list.append(compressed_output_np)


combined_compressed_output = np.concatenate(compressed_outputs_list, axis=0)

file_path_combined = 'MRL_FaceNet_IJB_C_output.txt'

# Write the combined array to a text file with floating-point format
np.savetxt(file_path_combined, combined_compressed_output, fmt='%.6f')

#Ignore Below

In [20]:
warnings.filterwarnings("ignore", category=UserWarning)


processedTensor = torch.zeros((len(testSet), 128))
identityLabel = torch.zeros((len(testSet)))
count = 0

for i,data in enumerate(testloader):
    
    inputs = data[0].to(device=device)
    iden_label = data[1]['name'].to(device=device)
    inputs = inputs[:, [2, 1, 0], :, :]    
    embeddings,_ = adaFaceModel(inputs)
    encoded, _ = aeModel(embeddings)
    # compressed_img1 = compressionModel1(embeddings)
    # compressed_img2 = compressionModel2(compressed_img1)
    # compressed = combinedModel(embeddings)
    processedTensor[count : count + embeddings.shape[0]] = torch.tensor(encoded)
    identityLabel[count:count + iden_label.shape[0]] = torch.tensor(iden_label)


    count = count + embeddings.shape[0]
    

In [21]:
identityAccuracy = 0

count = 0

while(count < processedTensor.shape[0]):

    currentFace = processedTensor[count : count + 1]
    cosine_similarity = nn.functional.cosine_similarity(currentFace, processedTensor)
    if(identityLabel[torch.topk(cosine_similarity, k = 2)[1][1]].item() == identityLabel[count].item()):
        identityAccuracy = identityAccuracy +1
    count = count + 1
    # print(count)
print(identityAccuracy/count)

0.8325


In [ ]:
# FAmodel.trainCompressionModel(trainloader,testloader,adaFaceModel,compressionModel, compressionLoss, device, 10)

In [ ]:
trainSetDet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv', get_pairs=False, transform=transform)
trainloaderDet = DataLoader(trainSetDet, batch_size=16, shuffle=True, num_workers=2)

testSetDet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/final_clebSET_test.csv', get_pairs=False, transform=transform)
testloaderDet = DataLoader(testSetDet, batch_size=16, shuffle=False, num_workers=2)


In [31]:
aeModel = torch.load('/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/CompressionCode/AeModel_20_bs32_82acc_best.pt')
aeModel.to(device)
FAmodel=faceAnalytics()
FAmodel.to(device)
aeModel.eval()
trainingAcc = FAmodel.trainModel(trainloader,testloader,adaFaceModel,aeModel, device,5)

Gender Accuracy: 0.9623919084766325 Age Acc: 0.6096150319513606  ethnAcc: 0.9649875156935491
Test Gender Accuracy: 0.934375 Test Age Accuracy: 0.571875 Test ethnicity Accuracy: 0.925625
Gender Accuracy: 0.969106631494308 Age Acc: 0.5989786849863872  ethnAcc: 0.9704608613466124
Gender Accuracy: 0.971490640296802 Age Acc: 0.5989786849863872  ethnAcc: 0.97325396041699
Test Gender Accuracy: 0.954375 Test Age Accuracy: 0.571875 Test ethnicity Accuracy: 0.921875
Gender Accuracy: 0.9732398538560284 Age Acc: 0.5989786849863872  ethnAcc: 0.9745235509035253


KeyboardInterrupt: 

In [ ]:
FAmodel.test(testloaderDet,adaFaceModel, compressionModel, device)

In [ ]:
torch.save(model,"arcFaceCelebSetBase.pt")

In [ ]:
torch.save(model,"negGenderBaseWiki.pt")

In [ ]:
plt.plot(trainingAcc)

In [ ]:
torch.save(model,"model92.pt")

In [ ]:
count=0
input1=[]
for i,data in enumerate(val_dataloader):
    
    if(count==0):
     inputs=resnet(data["image"].to(device))

 
     input1 = polyprotect(0,inputs[0])

     break
    count=count+1

In [ ]:
input1

In [ ]:
for param in model.parameters():
    print(param.shape)
    print(param[0])
    print(torch.dot(input1,param[1]))
    break 

In [ ]:
input1

In [ ]:
output_directory=""
file_name = "input1.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in input1:
            file.write(f"{value}\n")

In [ ]:
model=faceAnalytics()
model=torch.load("/home/csgrad/byalavar/FHE/HEAAN/modelUsing0.pt")
model.to(device)
model.test(dataloader,device)

In [ ]:
torch.save(model,"modelUsing0.pt")

In [ ]:
count=0
for param in model.parameters():
    print(param.shape)
    count=count+1
    if(count==2):
        print(param)

In [ ]:
a=nn.Linear(4,2)
input1=torch.rand((1,4))
print("input1",input1)
for param in a.parameters():
    print("param",param)
print(a(input1))


In [ ]:
model=faceAnalytics()
model=torch.load("modelUsing0.pt")
model.to(device)
count=0
ageBias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==7):
       ageBias = param.tolist()
    count=count+1
print(len(ageBias),len(ageBias[0]))


In [ ]:
ageBias

In [ ]:
bias2

In [ ]:
# Create a list of lists with shape (512, 128)


# Define the directory where you want to save the text files
output_directory = "ageWeights"

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

# Write each list to a separate text file


# Write each list to a separate text file
for i, sublist in enumerate(ageWeights):
    # Define the file name with leading zeros
    file_name = f"{i:03d}.txt"

    # Write each value in the sublist on a new line
    with open(os.path.join(output_directory, file_name), "w") as file:
        for value in sublist:
            file.write(f"{value}\n")

print("Files saved successfully.")


In [ ]:
count=1
layer1Bias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==2):
       layer1Bias = param.tolist()
       break
    count=count+1
print(len(layer1Bias),len(layer1Bias[0]))

In [ ]:
output_directory=""
file_name = "ageBias.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in ageBias:
            file.write(f"{value}\n")